In [2]:
import numpy as np
import plotly.graph_objects as go
import csv
from scipy import optimize
import sys,os

In [ ]:
def read_data(file_path):
    with open(file_path) as f:
        reader = csv.reader(f)
        data = []
        for column in reader:
            column[0] = column[0].replace('(', '').replace("'" , '')
            column[2] = column[2].replace(')', '').replace("'" , '')
            column[1] = column[1].replace("'" , '')
            column[0] = float(column[0])
            column[1] = float(column[1])
            column[2] = float(column[2])
            data.append(column)
            # print(data)
        data = np.array(data)
    return data
        # print(data)

In [ ]:
#Read energy in GeV of data files, save to list named energies
energies = []

#Read all files in data folder
for root, dir, files in os.walk('../data'):
    for file in files:
        if file.endswith('.csv'):
            energies.append(file.replace('.csv', '').replace('_', '.').replace('G', ''))
energies = np.array(energies, dtype=float)

In [ ]:
results = []
for root, dir, files in os.walk('../data'):
    for file in files:
        if file.endswith('.csv'):
            # print(file)
            data = read_data(os.path.join(root, file))
            results.append(data)

In [ ]:
for i, array in enumerate(results):
    print(f'data set{i} shape: {array.shape}')

In [ ]:
def plot_pos(arrays, title, directory):
    # Create the output directory if it doesn't already exist
    output_dir = "../media/plots/1.1_pion-/pos/"
    os.makedirs(output_dir, exist_ok=True)

    for i, array in enumerate(arrays):
        # np.savetxt(f'{directory}/array_{i}.csv', array, delimiter=',')
        # print(f"plotting array {array[:,0]}")
        energy_str = str(energies[i]).replace('.', '_')
        fig = go.Figure(data=go.Scatter(x=array[:,0], y=array[:,1], mode='markers'))
        fig.update_layout(title=f"{energy_str}"+ 'G', xaxis_title='x', yaxis_title='y')
        fig.update_layout(
            width=500,
            height=500,
            margin=dict(
                l=50,
                r=50,
                b=100,
                t=100,
                pad=4
            ),
            xaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1
            ),
            yaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1
            )
        )
        # fig.show()
        # fig.write_image('../media/plots/'+title+'.png')
        # Create the output directory if it doesn't already exist
        fig.write_image("../media/plots/1.1_pion-/pos/"+f"{energies[i]}"+ 'G'+".png")


        # print("../media/plots/"+f"{energy_str}"+ 'G'+".png")

In [ ]:
plot_dir = '../media/plots'
plot_pos(results, 'GeV', plot_dir)

# Calculate emision angle

In [ ]:
def momentum(E, m):
    return np.sqrt(E**2 - m**2)

In [ ]:
def fit_circle(data):
    x, y = data[:,0], data[:,1]
    x_m, y_m = np.mean(x), np.mean(y)

    def calc_R(xc, yc):
        """ calculate the distance of each 2D points from the center (xc, yc) """
        return np.sqrt((x-xc)**2 + (y-yc)**2)

    def f(c):
        """ calculate the algebraic distance between the data points and the mean circle centered at c=(xc, yc) """
        Ri = calc_R(*c)
        return Ri - Ri.mean()

    center_estimate = x_m, y_m
    center, _ = optimize.leastsq(f, center_estimate)

    xc, yc = center
    Ri       = calc_R(*center)
    R       = Ri.mean()
    return xc, yc, R

In [9]:
radii = []
xc=[]
yc=[]
momenta = []
# print(type(energies))
for e in range(len(energies)):
    p = momentum(energies[e], 0.139)
    print(energies[e])
    momenta.append(p)
momenta_array = np.array(momenta)
for array in results:
    radii.append(fit_circle(array)[2])
    xc.append(fit_circle(array)[0])
    yc.append(fit_circle(array)[1])
radii = np.array(radii)
xc = np.array(xc)
yc = np.array(yc)
print("momenta array", momenta_array.shape, "energies", energies.shape, "radios ",radii.shape)
# distancia 0.24 m
d = 240 # mm
cherenkov_angle = []
for i, array in enumerate(results):
    theta = np.arctan(radii[i]/d)
    cherenkov_angle.append(theta)
cherenkov_ang = np.array(cherenkov_angle)


processed_data = np.column_stack((energies, radii, momenta_array,cherenkov_ang))
print(processed_data.shape)

0.31
0.32
0.33
0.34
0.35
0.36
0.37
0.38
0.39
0.4
0.42
0.44
0.46
0.48
0.5
0.55
0.6
0.65
0.7
0.8
0.9
1.0
momenta array (22,) energies (22,) radios  (22,)
(22, 4)


In [10]:
# distancia 0.24 m
d = 240 # mm
cherenkov_angle = []
for i, array in enumerate(results):
    theta = np.arctan(radii[i]/d)
    cherenkov_angle.append(theta)
    print(f'angle for {energies[i]} GeV: {theta}')
cherenkov_ang = np.array(cherenkov_angle)

print(cherenkov_ang)

angle for 0.31 GeV: 0.07373454234081654
angle for 0.32 GeV: 0.12917491914285256
angle for 0.33 GeV: 0.16385026266046682
angle for 0.34 GeV: 0.19194417008870351
angle for 0.35 GeV: 0.21409122503908318
angle for 0.36 GeV: 0.23258316987153793
angle for 0.37 GeV: 0.24967447458425057
angle for 0.38 GeV: 0.26319099687811115
angle for 0.39 GeV: 0.29221497284629944
angle for 0.4 GeV: 0.2873830282242025
angle for 0.42 GeV: 0.30589538697248525
angle for 0.44 GeV: 0.3224455656869055
angle for 0.46 GeV: 0.3348887720176852
angle for 0.48 GeV: 0.34584427476650453
angle for 0.5 GeV: 0.3553270529265008
angle for 0.55 GeV: 0.3927056177169974
angle for 0.6 GeV: 0.38791223815463727
angle for 0.65 GeV: 0.39901760959037924
angle for 0.7 GeV: 0.4068464238310392
angle for 0.8 GeV: 0.41927032029829564
angle for 0.9 GeV: 0.4265022727168366
angle for 1.0 GeV: 0.4322450045552253
[0.07373454 0.12917492 0.16385026 0.19194417 0.21409123 0.23258317
 0.24967447 0.263191   0.29221497 0.28738303 0.30589539 0.32244557
 

In [11]:
def plot_circle(xc, yc, radii, arrays, energies):
    # Create the output directory if it doesn't already exist
    output_dir = "../media/plots/1.1_pion-/fit/"
    os.makedirs(output_dir, exist_ok=True)
    for i, array in enumerate(arrays):
        # Create the output directory if it doesn't already exist
        output_dir = "../media/plots/1.1_pion-/fit/"
        os.makedirs(output_dir, exist_ok=True)

        energy_str = str(energies[i]).replace('.', '_')
        theta_fit = np.linspace(0, 2*np.pi, 180)
        x_fit = xc[i] + (radii[i])*np.cos(theta_fit)
        y_fit = yc[i] + (radii[i])*np.sin(theta_fit)
        fig = go.Figure(data=go.Scatter(x=x_fit, y=y_fit, mode='lines', marker=dict(color='blue', opacity=0.8)))
        fig.add_trace(go.Scatter(x=array[:,0], y=array[:,1], mode='markers', marker=dict(color='violet', size=3, opacity=0.8)))
        fig.update_layout(title=f"{energy_str}"+ 'G', xaxis_title='y', yaxis_title='z')

        fig.update_layout(
            width=500,
            height=500,
            margin=dict(
                l=50,
                r=50,
                b=100,
                t=100,
                pad=4
            ),
            xaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1,
                scaleanchor="x",
                range=[-200, 200]
            ),
            yaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1,
                scaleanchor="y",
                range=[-200, 200]
            )
        )
        #fig.show()
        fig.write_image("../media/plots/1.1_pion-/fit/"+f"{energies[i]}"+ 'G'+".png")



In [ ]:
plot_circle(xc, yc, radii, results, energies)

In [12]:
def plot_momenta_cherenkov_angle(data):
    # Create the output directory if it doesn't already exist
    output_dir = "../media/plots/1.1_pion-/p/"
    os.makedirs(output_dir, exist_ok=True)
    fig = go.Figure(data=go.Scatter(x=data[:,2], y=data[:,3], mode='markers', marker=dict(color='darkgreen', opacity=0.8, size=2)))
    fig.update_layout(title='Cherenkov angle vs momenta', xaxis_title='momenta', yaxis_title='cherenkov angle', width=500, height=500)
    fig.write_image("../media/plots/1.1_pion-/p/"+"n_1_1"+".png")

In [ ]:
plot_momenta_cherenkov_angle(processed_data)

In [ ]:
def get_beta(energy, momentum):
    beta = []
    for i in range(len(energy)):
        beta.append(momentum[i]/energy[i])
    return np.array(beta)

In [ ]:
beta = get_beta(energies, momenta)
cos_beta = np.cos(beta)
v = 1/beta
print(v.shape, cos_beta.shape)

In [ ]:
def plot_cos_beta(cos,v):
    # Create the output directory if it doesn't already exist
    output_dir = "../media/plots/1.1_pion-/beta/"
    fig = go.Figure(data=go.Scatter(x=cos, y=v, mode='markers', marker=dict(color='darkgreen', opacity=0.8, size=2)))
    fig.update_layout(title='Cherenkov angle vs beta', xaxis_title='beta', yaxis_title='cherenkov angle', width=500, height=500)
    fig.write_image("../media/plots/1.1_pion-/beta/"+"n_1_1"+".png")

In [16]:
plot_cos_beta(cos_beta, v)

KeyboardInterrupt: 